<a href="https://colab.research.google.com/github/dangdang2222/LG_Aimers/blob/main/%EB%B6%84%EB%A5%98%2B%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [160]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
train = pd.read_csv('./drive/MyDrive/LG_Aimer/train.csv')
test = pd.read_csv('./drive/MyDrive/LG_Aimer/test.csv')
meta_x_features= pd.read_csv('./drive/MyDrive/LG_Aimer/meta/x_feature_info.csv')
meta_y_features= pd.read_csv('./drive/MyDrive/LG_Aimer/meta/y_feature_info.csv')

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
import random
import os
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    np.random.seed(seed)
seed_everything(42)

train_df = pd.read_csv('./drive/MyDrive/LG_Aimer/train.csv')
drop_sensors = ['ID','X_04', 'X_23', 'X_47', 'X_48']
train_df= train_df.drop(labels = drop_sensors,axis=1, inplace=False)
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

In [161]:
df = pd.read_csv('./drive/MyDrive/LG_Aimer/train.csv')
df1 = df.copy()
df1=df[(df['Y_01']>=0.2)&(df['Y_01']<2.0)&(df['Y_02']>=0.2)
&(df['Y_02']<=2.1)&(df['Y_02']>=0.2)
&(df['Y_03']>=0.2)&(df['Y_03']<=2.1)
&(df['Y_04']<=19)&(df['Y_04']>=7)&(df['Y_05']<=36.5)&(df['Y_05']>=22)
&(df['Y_06']>=-19.2)&(df['Y_06']<=19)&(df['Y_07']>=2.4)&(df['Y_07']<=4)
&(df['Y_08']>=-29.2)&(df['Y_08']<=-24)&(df['Y_09']>=-29.2)&(df['Y_09']<=-24)
&(df['Y_10']>=-30.6)&(df['Y_10']<=-20)&(df['Y_11']>=19.6)&(df['Y_11']<=26.6)
&(df['Y_12']>=-29.2)&(df['Y_12']<=-24)&(df['Y_13']>=-29.2)&(df['Y_13']<=-24)
&(df['Y_14']>=-29.2)&(df['Y_14']<=-24)]

for idx in range(0,39607):
    if idx in df1.index:
        df.loc[idx,'pass']=1
    else:
        df.loc[idx,'pass']=0

In [162]:
from sklearn.model_selection import train_test_split

drop_sensors = ['ID','X_04', 'X_23', 'X_47', 'X_48','X_14','X_15','X_16','X_17','X_18','Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06', 'Y_07', 'Y_08', 'Y_09',
       'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']
df= df.drop(labels = drop_sensors,axis=1, inplace=False)
label = df['pass']
df= df.drop(labels = 'pass',axis=1, inplace=False)
print(len(df.columns))
x_train, x_test, y_train, y_test = train_test_split(df, label, test_size=0.3, random_state=10)

47


In [163]:
x_train = x_train.to_numpy().reshape(x_train.shape[0], x_train.shape[1],1)
print(x_train.shape)
x_test = x_test.to_numpy().reshape(x_test.shape[0],x_test.shape[1],1)
print(x_test.shape)

(27724, 47, 1)
(11883, 47, 1)


## DNN 분류기

In [21]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input, Bidirectional,TimeDistributed, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras import optimizers

model = Sequential()
model.add(Input(shape=(47,1)))
model.add(Dense(256,activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 47, 256)           512       
                                                                 
 dense_7 (Dense)             (None, 47, 128)           32896     
                                                                 
 dense_8 (Dense)             (None, 47, 64)            8256      
                                                                 
 flatten_1 (Flatten)         (None, 3008)              0         
                                                                 
 dense_9 (Dense)             (None, 128)               385152    
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 1)                

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 30, verbose = 1)
hist = model.fit(x_train, y_train, epochs = 1000, batch_size = 20, validation_data = (x_test, y_test),callbacks = [early_stopping])

In [164]:
preds = model.predict(x_test)

In [165]:
predss = []

count = 0 
for i in preds:
  if i < 0.907: #임계값 바꾸기
    count+=1
    predss.append(0)
  else:
    predss.append(1)
print(count)

1162


In [166]:
x_test.shape

(11883, 47, 1)

In [167]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predss)
print(cm)

[[ 119 1040]
 [1043 9681]]


In [168]:
test = pd.read_csv('./drive/MyDrive/LG_Aimer/test.csv')
drop_sensors = ['ID','X_04', 'X_23', 'X_47', 'X_48','X_14','X_15','X_16','X_17','X_18']
test = test.drop(labels = drop_sensors,axis=1, inplace=False)
test = test.to_numpy().reshape(test.shape[0], test.shape[1],1)
print(test.shape)

(39608, 47, 1)


In [169]:
preds = model.predict(test)

In [170]:
predss = []

count = 0 
for i in preds:
  if i < 0.907: #임계값 바꾸기
    count+=1
    predss.append(0)
  else:
    predss.append(1)
print(count)

4060


## pass 회귀 모델



In [88]:
train_x = train.filter(regex='X') # Input : X Feature
train_y = train.filter(regex='Y') # Input : y Feature
df=train_y
df1=df[(df['Y_01']>=0.2)&(df['Y_01']<2.0)&(df['Y_02']>=0.2)
&(df['Y_02']<=2.1)&(df['Y_02']>=0.2)
&(df['Y_03']>=0.2)&(df['Y_03']<=2.1)
&(df['Y_04']<=19)&(df['Y_04']>=7)&(df['Y_05']<=36.5)&(df['Y_05']>=22)
&(df['Y_06']>=-19.2)&(df['Y_06']<=19)&(df['Y_07']>=2.4)&(df['Y_07']<=4)
&(df['Y_08']>=-29.2)&(df['Y_08']<=-24)&(df['Y_09']>=-29.2)&(df['Y_09']<=-24)
&(df['Y_10']>=-30.6)&(df['Y_10']<=-20)&(df['Y_11']>=19.6)&(df['Y_11']<=26.6)
&(df['Y_12']>=-29.2)&(df['Y_12']<=-24)&(df['Y_13']>=-29.2)&(df['Y_13']<=-24)
&(df['Y_14']>=-29.2)&(df['Y_14']<=-24)]
len(df1.index)


for idx in range(0,39607):
    if idx in df1.index:
        train_x.loc[idx,'X_60']=1
    else:
        train_x.loc[idx,'X_60']=0
train_x['X_60'].value_counts()

df1 = train_x.query('X_60 ==1')
df2= train_x.query('X_60==0')

df1_target= train_y.loc[df1.index,:]
df2_target = train_y.drop(index=df1.index)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [89]:
drop_x_cols = ['X_04', 'X_23', 'X_47', 'X_48','X_14','X_15','X_16','X_17','X_18','X_60']
df1=df1.drop(columns=drop_x_cols)
df2=df2.drop(columns=drop_x_cols)

In [90]:
from sklearn.metrics import mean_squared_error
def nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,13): # ignore 'ID'
        rmse = mean_squared_error(np.array(gt)[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(np.array(gt)[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [91]:

from sklearn.model_selection import train_test_split
x_train_pass, x_val_pass, y_train_pass, y_val_pass = train_test_split(df1, df1_target, 
test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler

import xgboost as xgb
xgb_1 = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma = 1, subsample=0.75, colsample_bytree = 1, max_depth=7)).fit(x_train_pass, y_train_pass)
preds_1 = xgb_1.predict(x_val_pass)
score1=nrmse(y_val_pass,preds_1)

[12:47:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:47:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:48:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:48:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:48:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:49:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [92]:
score1

1.674105697989075

## 전체 회귀 모델

In [95]:
train_df = pd.read_csv('./drive/MyDrive/LG_Aimer/train.csv')
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

drop_x_cols = ['X_04', 'X_23', 'X_47', 'X_48','X_14','X_15','X_16','X_17','X_18']
train_x=train_x.drop(columns=drop_x_cols)

In [97]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train_x, train_y, 
test_size=0.2, random_state=42)

import xgboost as xgb
xgb_2 = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma = 1, subsample=0.75, colsample_bytree = 1, max_depth=7)).fit(x_train, y_train)
preds_2 = xgb_2.predict(x_val)
score2=nrmse(y_val,preds_2)

[12:52:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:52:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:52:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:52:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:53:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:53:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:53:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:53:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [98]:
score2

1.9193847169763514

## testing

In [171]:
test = pd.read_csv('./drive/MyDrive/LG_Aimer/test.csv')
drop_sensors = ['ID','X_04', 'X_23', 'X_47', 'X_48','X_14','X_15','X_16','X_17','X_18']
test = test.drop(labels = drop_sensors,axis=1, inplace=False)

In [172]:
test['pass'] = predss

In [173]:
test

,X_01,X_02,X_03,X_05,X_06,X_07,X_08,X_09,X_10,X_11,...,X_46,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56,pass
0,68.504,103.321,76.67,101.867,73.963,30.51,63.57,239.80,0.0,0.0,...,1469,17227.63,138.130429,129.460682,141.506570,133.427229,129.711498,133.138096,121.859684,1
1,67.485,103.320,69.37,101.992,67.845,28.03,116.99,189.23,0.0,0.0,...,1462,17134.53,136.148839,128.266277,145.911745,131.196417,132.411480,133.629025,124.178623,1
2,69.524,103.320,68.97,101.884,77.022,29.65,205.68,214.93,0.0,0.0,...,1469,14860.83,120.447446,119.988804,132.099908,120.450155,130.051708,128.252972,114.475628,1
3,69.524,103.320,65.87,101.866,73.963,28.15,103.38,180.80,0.0,0.0,...,1469,15252.53,133.994695,125.069180,147.507669,123.142653,125.963665,139.666592,126.589253,1
4,73.603,103.321,66.67,101.891,74.983,29.92,71.20,231.93,0.0,0.0,...,1469,10752.23,137.918202,135.116192,138.600473,127.173033,137.252712,134.411335,124.020016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,68.504,103.320,63.97,103.157,68.864,29.49,116.35,284.16,0.0,0.0,...,1469,62123.53,127.741246,126.494312,139.119905,125.271109,128.284572,140.176945,128.292843,1
39604,68.504,103.320,61.37,103.137,68.864,32.29,116.28,272.41,0.0,0.0,...,1469,61844.13,127.767377,124.062809,138.238664,119.879393,127.322529,137.312047,131.570614,1
39605,69.524,103.320,63.67,103.149,69.884,30.00,113.05,295.54,0.0,0.0,...,1469,60277.53,128.593640,124.774037,138.659624,123.999571,126.075542,135.656132,127.671108,1
39606,67.485,103.321,61.77,103.148,67.845,32.05,115.05,267.26,0.0,0.0,...,1469,60236.73,121.110646,125.471699,134.989984,120.889578,129.296909,132.673977,131.882893,1


In [132]:
pass_test = test[test['pass']==1]
fail_test = test[test['pass']==0]

In [135]:
pass_test = pass_test.drop('pass',axis=1)

In [137]:
fail_test = fail_test.drop('pass',axis=1)

In [138]:
pass_pred = xgb_1.predict(pass_test)

In [140]:
pass_pred = pd.DataFrame(pass_pred, index=pass_test.index)

In [146]:
fail_pred = xgb_2.predict(fail_test)

In [147]:
fail_pred = pd.DataFrame(fail_pred, index=fail_test.index)

In [152]:
final = pd.concat([pass_pred, fail_pred]).sort_index(ascending=True)

In [174]:
final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.388842,1.102641,1.063192,14.275603,31.500191,16.832821,3.110303,-26.066635,-25.971573,-22.290552,24.437771,-26.009293,-25.985958,-26.050285
1,1.467951,1.208646,1.099824,13.711733,31.361696,16.735426,3.155328,-26.085487,-26.147621,-22.215969,24.387421,-26.023899,-25.966240,-26.084509
2,1.379874,1.120898,0.995068,13.565459,31.922886,16.795238,2.995768,-25.933990,-25.828384,-21.997131,24.602486,-25.424952,-25.782543,-25.691729
3,1.417747,1.128300,1.016971,15.474136,32.809299,17.239452,3.016834,-25.594851,-25.730661,-21.608120,24.933098,-25.582399,-25.713903,-25.521070
4,1.323217,0.982340,0.953654,15.327987,32.244492,17.248947,2.989896,-25.579590,-25.564873,-21.784492,25.224924,-25.536865,-25.360033,-25.689516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,1.321902,1.010210,1.043739,13.037948,31.198471,16.769083,3.141090,-26.453703,-26.471933,-22.634325,24.489698,-26.368305,-26.424488,-26.454170
39604,1.257241,0.920144,0.986886,14.857346,30.955332,16.887556,3.130708,-26.411585,-26.364214,-22.843750,24.476746,-26.261854,-26.247116,-26.320038
39605,1.315065,0.946337,0.922454,13.354900,30.850128,16.626284,3.178425,-26.397606,-26.540842,-22.798115,24.201654,-26.559097,-26.448717,-26.577545
39606,1.257449,0.933694,1.021127,13.207811,30.962023,16.702501,3.125632,-26.436462,-26.405931,-22.777399,24.479591,-26.467655,-26.488829,-26.458929


In [155]:
final_p = final.to_numpy() 

In [156]:
submit = pd.read_csv('./drive/MyDrive/LG_Aimer/sample_submission.csv') #/drive/MyDrive/LG_Aimer
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = final_p[:,idx-1]
print('Done.')
submit.to_csv('./submit.csv', index=False)

Done.
